# TorchFort example train_distributed

2025-06-20

Running on SDumont

Original file: /examples/fortran/simulation/train_distributed.f90

The "train_distributed.f90" is compiled during container creation, as part of the package.

There is also a new version: train_distrib-sd-slurm-02.ipynb

In [3]:
%cd /scratch/ampemi/eduardo.miranda2/radnn/torchfort/repository/examples/fortran/simulation

/scratch/ampemi/eduardo.miranda2/radnn/torchfort/repository/examples/fortran/simulation


In [1]:
%%bash
singularity exec --nv /scratch/ampemi/eduardo.miranda2/torchfort.sif \
    mpiexec --version

mpiexec (OpenRTE) 4.1.7a1

Report bugs to http://www.open-mpi.org/community/help/


In [ ]:
#123456789 123456789 123456789 123456789 123456789 123456789 123456789 12

Running on GPU:

In [4]:
%%bash
singularity exec --nv /scratch/ampemi/eduardo.miranda2/torchfort.sif \
    bash -c "mpirun --allow-run-as-root -np 2 ./train_distributed"

 Run settings:
 	configfile: config_mlp_native.yaml
 	simulation_device: gpu
 	train_device: gpu
 	checkpoint_dir:NONE
 	output_model_name: model.pt
 	output_checkpoint_dir: checkpoint
 	ntrain_steps:        100000
 	nval_steps:          1000
 	val_write_freq:            10
 
TORCHFORT::WARN: enable_wandb_hook is true but the environment variable TORCHFORT_LOGDIR was not specified. To enable logging, set TORCHFORT_LOGDIR to a writeable directory prior to launching wandb_helper.py and your torchfort enabled application.
TORCHFORT::WARN: enable_wandb_hook is true but the environment variable TORCHFORT_LOGDIR was not specified. To enable logging, set TORCHFORT_LOGDIR to a writeable directory prior to launching wandb_helper.py and your torchfort enabled application.
 start training...
TORCHFORT::INFO: model: mymodel, step_train: 100, loss: 0.310131, lr: 0.000999998
TORCHFORT::INFO: model: mymodel, step_train: 200, loss: 5.62055, lr: 0.00099999
TORCHFORT::INFO: model: mymodel, step_train: 3

Running on CPU:

In [5]:
%%bash
singularity exec --nv /scratch/ampemi/eduardo.miranda2/torchfort.sif \
    bash -c "mpirun --allow-run-as-root -np 2 \
                    --simulation_device -1 \
                    --train_device -1 \
                    ./train_distributed"

 Run settings:
 	configfile: config_mlp_native.yaml
 	simulation_device: cpu
 	train_device: cpu
 	checkpoint_dir:NONE
 	output_model_name: model.pt
 	output_checkpoint_dir: checkpoint
 	ntrain_steps:        100000
 	nval_steps:          1000
 	val_write_freq:            10
 
TORCHFORT::WARN: enable_wandb_hook is true but the environment variable TORCHFORT_LOGDIR was not specified. To enable logging, set TORCHFORT_LOGDIR to a writeable directory prior to launching wandb_helper.py and your torchfort enabled application.
TORCHFORT::WARN: enable_wandb_hook is true but the environment variable TORCHFORT_LOGDIR was not specified. To enable logging, set TORCHFORT_LOGDIR to a writeable directory prior to launching wandb_helper.py and your torchfort enabled application.
 start training...
TORCHFORT::INFO: model: mymodel, step_train: 100, loss: 0.311064, lr: 0.000999998
TORCHFORT::INFO: model: mymodel, step_train: 200, loss: 5.69758, lr: 0.00099999
TORCHFORT::INFO: model: mymodel, step_train: 3

## SRUN

In [22]:
%%writefile train_distributed.srm
#!/bin/bash
#SBATCH --partition=sequana_gpu_dev   # SLURM_JOB_PARTITION
#SBATCH --job-name=train_distributed  # SLURM_JOB_NAME
#SBATCH --nodes=1                     # SLURM_JOB_NUM_NODES
#SBATCH --ntasks-per-node=2           # SLURM_NTASKS_PER_NODE
    # Not compatible with the --cpus-per-gpu option:
    # #SBATCH --cpus-per-task=1             # SLURM_CPUS_PER_TASK
#SBATCH --gpus=2                      # SLURM_GPUS
#SBATCH --gpus-per-node=2             # SLURM_GPUS_PER_NODE
    # #SBATCH --cpus-per-gpu=1              # SLURM_CPUS_PER_GPU
#SBATCH --distribution=block:cyclic   # SLURM_DISTRIBUTION
#SBATCH --time=00:10:00               # Limit execution time
# SLURM output environment variables:
# https://slurm.schedmd.com/sbatch.html#SECTION_OUTPUT-ENVIRONMENT-VARIABLES
echo '======================================='
echo '- Job ID:' $SLURM_JOB_ID
echo '- Nº of nodes in the job:' $SLURM_JOB_NUM_NODES
echo '- Nº of tasks per node:' $SLURM_NTASKS_PER_NODE
echo '- Nº of tasks:' $SLURM_NTASKS
echo '- Nº of cpus per task:' $SLURM_CPUS_PER_TASK
echo '- Partition:' $SLURM_JOB_PARTITION
echo '- Dir from which sbatch was invoked:' ${SLURM_SUBMIT_DIR##*/}
echo -n '- Nodes allocated to the job: '
nodeset -e $SLURM_JOB_NODELIST
echo '----------------------------------------'
#cd $SLURM_SUBMIT_DIR
cd /scratch/ampemi/eduardo.miranda2/radnn/torchfort/repository/examples/fortran/simulation
export SLURM_MPI_TYPE=pmi2
#export OMPI_HOME=$(dirname $(dirname $(which mpirun)))
#module load openmpi/gnu/4.1.6
module load openmpi/gnu/4.1.4+gcc-12.4+cuda-11.6_sequana
echo -n '<1. starting python script > ' && date
echo '-- output -----------------------------'

srun \
singularity exec --nv /scratch/ampemi/eduardo.miranda2/torchfort.sif \
    bash -c " mpirun --allow-run-as-root \
                     ./train_distributed"

echo '-- end --------------------------------'
echo -n '<2. quit>                    ' && date

Overwriting train_distributed.srm


In [23]:
cd /scratch/ampemi/eduardo.miranda2/radnn/torchfort/repository/examples/fortran/simulation/
import time
a = ! sbatch train_distributed.srm
time.sleep(10)
! squeue --me --name=train_distributed
while True:
    time.sleep(10)
    b = ! squeue --me --name=train_distributed
    if len(b) < 2 : break
cat /scratch/ampemi/eduardo.miranda2/radnn/torchfort/repository/examples/fortran/simulation/slurm-{a[0].replace('Submitted batch job ','')}.out

/scratch/ampemi/eduardo.miranda2/radnn/torchfort/repository/examples/fortran/simulation
             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
          11351756 sequana_g train_di eduardo.  R       0:09      1 sdumont8046
- Job ID: 11351756
- Nº of nodes in the job: 1
- Nº of tasks per node: 2
- Nº of tasks: 2
- Nº of cpus per task:
- Partition: sequana_gpu_dev
- Dir from which sbatch was invoked: simulation
- Nodes allocated to the job: sdumont8046
----------------------------------------
<1. starting python script > sáb jun 14 11:35:49 -03 2025
-- output -----------------------------
 Run settings:
 	configfile: config_mlp_native.yaml
 	simulation_device: gpu
 	train_device: gpu
 	checkpoint_dir:NONE
 	output_model_name: model.pt
 	output_checkpoint_dir: checkpoint
 	ntrain_steps:        100000
 	nval_steps:          1000
 	val_write_freq:            10
 
 Run settings:
 	configfile: config_mlp_native.yaml
 	simulation_device: gpu
 	train_device: gpu
 	

## Debugging of old version

There were basically two problems happening together. The first is the use of an Nvidia Container with Nvidia tools that forces, in this case, the use of nodes and queues containing GPU, even if the GPU is not used. The second is the configuration of the Slurm configuration script, which now after the system expansion requires the use of new options like "--cpus-per-gpu" that conflict with existing options like "--cpus-per-task".

---
## Why Removing CPU Binding Options Fixed The Slurm Job

The job ran successfully after removing the lines

    # #SBATCH --cpus-per-task=1             # SLURM_CPUS_PER_TASK
    # #SBATCH --cpus-per-gpu=1              # SLURM_CPUS_PER_GPU

Let's break down why eliminating `#SBATCH --cpus-per-task=1` and `#SBATCH --cpus-per-gpu=1` likely resolved the issue, especially considering the `hwloc_set_cpubind` error you were seeing.

---
## Understanding Slurm Job Failures: The Case of CPU Binding Conflicts

A common issue encountered when running High-Performance Computing (HPC) jobs, particularly those utilizing MPI (Message Passing Interface) with GPUs and container technologies like Singularity, is a conflict in **CPU binding**. This often manifests as an `hwloc_set_cpubind returned "Error"` message in the job output.

---
### The Error Message Explained

The error `hwloc_set_cpubind returned "Error" for bitmap "1"` is highly indicative of a CPU binding conflict.

* **`hwloc` (Hardware Locality):** This is a library used by MPI implementations (such as Open MPI) to understand the underlying hardware topology of the compute node, including CPU cores, memory, and accelerators (like GPUs). It's crucial for optimizing process placement and communication.
* **`cpubind` (CPU Binding):** This refers to the operation of "pinning" a process to specific CPU cores. Binding can improve performance by ensuring a process runs on designated cores, which can enhance cache utilization and reduce overhead from core migration.
* **`bitmap "1"`:** In the context of MPI, this often refers to the second process (rank 1, if using 0-indexed ranks). The error signifies that the attempt to bind this particular MPI process to a CPU core failed.

---
### Root Cause: Overlapping Binding Mechanisms

The fundamental reason for this error is usually an **overlap or conflict between different entities attempting to manage and bind processes to CPU cores simultaneously.**

1.  **Slurm's Resource Allocation and Binding:**
    * When an `--cpus-per-task` or `--cpus-per-gpu` directive is included in a Slurm batch script, it instructs the Slurm workload manager to allocate a specific number of CPU cores for each task or GPU and potentially to enforce a **binding** of those tasks to the allocated cores.

2.  **MPI's Internal Binding (via hwloc):**
    * MPI runtimes, like Open MPI (often invoked via `mpirun` or `srun`), have their own sophisticated mechanisms for **mapping and binding** MPI processes to CPU cores. They leverage libraries such as `hwloc` to perform this core binding, aiming to optimize inter-process communication and overall application performance.

**The Conflict:** When both Slurm and the MPI runtime try to bind processes to CPU cores independently, a collision can occur. Slurm might place a process on a set of cores, and then the MPI runtime might attempt to re-bind the same process to a different set of cores, or to the same cores in a conflicting manner. This second, conflicting binding request is then denied by the operating system or the container environment (e.g., Singularity), leading to the `hwloc_set_cpubind` error and job termination.

---
### The Solution: Consolidating CPU Binding Control

The resolution for this type of conflict typically involves **allowing a single entity to control the CPU binding** or ensuring that the binding instructions from different entities do not overlap in a detrimental way.

In the described scenario, removing the Slurm directives that explicitly enforce CPU binding (`#SBATCH --cpus-per-task=1` and `#SBATCH --cpus-per-gpu=1`) allowed the MPI runtime (and its `hwloc` integration) to take full control. Without Slurm's strict CPU binding directives, Open MPI could perform its process-to-core binding without encountering conflicts, resulting in a successful job launch.

By default, without explicit `cpus-per-task` or `cpus-per-gpu` options, Slurm typically assigns one CPU core per task (`--ntasks-per-node`). For many GPU-intensive applications, one CPU core per GPU-enabled process is sufficient to manage the GPU's operations and data transfer, as the primary computational bottleneck usually resides on the GPU itself.

---
### Key Takeaway for Future Jobs

When encountering `hwloc_set_cpubind` errors in MPI jobs, especially with GPUs and containers:

* **Review CPU Binding Directives:** Examine both your Slurm script and any environment variables or command-line arguments passed to your MPI launcher (`mpirun`, `srun`) within the container.
* **Prioritize One Binding Source:** It's often best to let **either** the Slurm workload manager **or** the MPI runtime handle the CPU core binding, rather than having both attempt to do so independently. Removing overly specific CPU binding options from Slurm is often the first step to resolving such conflicts.

## The error (history)

In [16]:
%cd /scratch/ampemi/eduardo.miranda2/radnn/torchfort/repository/examples/fortran/simulation/
import time
a = ! sbatch train_distributed.srm
print(a[0])
while True:
    time.sleep(10)
    b = ! squeue --user=$USER --name=train_distributed
    if len(b) < 2 : break
%cat /scratch/ampemi/eduardo.miranda2/radnn/torchfort/repository/examples/fortran/simulation/slurm-{a[0].replace('Submitted batch job ','')}.out

/scratch/ampemi/eduardo.miranda2/radnn/torchfort/repository/examples/fortran/simulation
Submitted batch job 11342498
- Job ID: 11342498
- Nº of nodes in the job: 1
- Nº of tasks per node: 2
- Nº of tasks: 2
- Nº of cpus per task: 1
- Partition: sequana_cpu_dev
- Dir from which sbatch was invoked: simulation
- Nodes allocated to the job: sdumont6167
----------------------------------------
<1. starting python script > sex jun  6 10:51:51 -03 2025
-- output -----------------------------
 Run settings:
 	configfile: config_mlp_native.yaml
 	simulation_device: cpu
 	train_device: cpu
 	checkpoint_dir:NONE
 	output_model_name: model.pt
 	output_checkpoint_dir: checkpoint
 	ntrain_steps:        100000
 	nval_steps:          1000
 	val_write_freq:            10
 
TORCHFORT::WARN: enable_wandb_hook is true but the environment variable TORCHFORT_LOGDIR was not specified. To enable logging, set TORCHFORT_LOGDIR to a writeable directory prior to launching wandb_helper.py and your torchfort enable

## Another try

In [14]:
%cd /scratch/ampemi/eduardo.miranda2/radnn/torchfort/repository/examples/fortran/simulation/
import time
a = ! sbatch train_distributed.srm
print(a[0])
while True:
    time.sleep(10)
    b = ! squeue --user=$USER --name=train_distributed
    if len(b) < 2 : break
%cat /scratch/ampemi/eduardo.miranda2/radnn/torchfort/repository/examples/fortran/simulation/slurm-{a[0].replace('Submitted batch job ','')}.out

/scratch/ampemi/eduardo.miranda2/radnn/torchfort/repository/examples/fortran/simulation
Submitted batch job 11342495
- Job ID: 11342495
- Nº of nodes in the job: 1
- Nº of tasks per node: 2
- Nº of tasks: 2
- Nº of cpus per task: 1
- Partition: sequana_cpu_dev
- Dir from which sbatch was invoked: simulation
- Nodes allocated to the job: sdumont6167
----------------------------------------
<1. starting python script > sex jun  6 10:50:41 -03 2025
-- output -----------------------------
 Run settings:
 	configfile: config_mlp_native.yaml
 	simulation_device: cpu
 	train_device: cpu
 	checkpoint_dir:NONE
 	output_model_name: model.pt
 	output_checkpoint_dir: checkpoint
 	ntrain_steps:        100000
 	nval_steps:          1000
 	val_write_freq:            10
 
TORCHFORT::WARN: enable_wandb_hook is true but the environment variable TORCHFORT_LOGDIR was not specified. To enable logging, set TORCHFORT_LOGDIR to a writeable directory prior to launching wandb_helper.py and your torchfort enable

- Job ID: 11342493
- Nº of nodes in the job: 1
- Nº of tasks per node: 2
- Nº of tasks: 2
- Nº of cpus per task: 1
- Partition: sequana_cpu_dev
- Dir from which sbatch was invoked: simulation
- Nodes allocated to the job: sdumont6167
----------------------------------------
<1. starting python script > sex jun  6 10:40:41 -03 2025
-- output -----------------------------
 Run settings:
 	configfile: config_mlp_native.yaml
 	simulation_device: cpu
 	train_device: cpu
 	checkpoint_dir:NONE
 	output_model_name: model.pt
 	output_checkpoint_dir: checkpoint
 	ntrain_steps:        100000
 	nval_steps:          1000
 	val_write_freq:            10
 
 Run settings:
 	configfile: config_mlp_native.yaml
 	simulation_device: cpu
 	train_device: cpu
 	checkpoint_dir:NONE
 	output_model_name: model.pt
 	output_checkpoint_dir: checkpoint
 	ntrain_steps:        100000
 	nval_steps:          1000
 	val_write_freq:            10
 
TORCHFORT::WARN: enable_wandb_hook is true but the environment variable T